# Trend Following

## Basic Setting

In [ ]:
import numpy as np
import pandas as pd

import plotly
import plotly.graph_objs as go

import rqdatac as rq
from rqdatac import *

from trend_following_objects import MarketInfo, ParameterData, RegimeInfo
from trend_following_strategy import EstimationParameter

In [26]:
def configure_plotly_browser_state():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

import IPython

IPython.get_ipython().events.register('pre_run_cell', configure_plotly_browser_state)

In [6]:
mkt_info = MarketInfo("510050", 0.03, 0.001, 0.001)

In [7]:
mkt_info.upper_boundary

0.0009995003330834232

In [8]:
mkt_info.lower_boundary

-0.0010005003335835344

In [9]:
para_info = ParameterData("510050")

In [10]:
para_info.bull_lambda

0

In [11]:
rq.initialized()

False

## Parameter Estimation

In [12]:
regime_info = RegimeInfo("000300", 0.2, 0.2)

In [13]:
import tushare as ts

ts.set_token('c3450fdabd780ea8ef13ebdd0230c2e52a5737b05cfb0377eecc2f84')
pro = ts.pro_api()

In [44]:
cons = ts.get_apis()
csi_300 = ts.bar('000300', conn=cons, asset='INDEX', start_date='2005-04-08', end_date='2020-01-01')
csi_300.index = pd.to_datetime(csi_300.index)
csi_300['date'] = csi_300.index
csi_300 = csi_300.sort_values(by = 'date')
csi_300['log_rtn'] = np.log(csi_300['close']).diff()
csi_300['rtn'] = (csi_300['close'] - csi_300['close'].shift())/csi_300['close']
csi_300.head()

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


AttributeError: 'NoneType' object has no attribute 'index'

In [15]:
est_para = EstimationParameter(csi_300, "close", regime_info)

In [16]:
print(est_para)

Estimate Parameters for 000300


In [23]:
est_para.find_bull_bear("bear")
market_regimes = est_para.get_regimes()

In [19]:
para_info = est_para.get_para_info("bull")
para_info

ParameterData(vt_symbol='000300', bull_mu=0.7286205497763798, bear_mu=-0.6640537455282965, bull_sigma=0.2510420807552449, bear_sigma=0.2979106817115861, constant_sigma=0.2682323013593184, bull_lambda=5.222781428336091, bear_lambda=4.59069816025367)

ParameterData(vt_symbol='000300', bull_mu=0.7286205497763798, bear_mu=-0.6640537455282965, bull_sigma=0.2510420807552449, bear_sigma=0.2979106817115861, constant_sigma=0.2682323013593184, bull_lambda=5.222781428336091, bear_lambda=4.59069816025367)


In [24]:
market_regimes

[{'state': 'bear',
  'start_date': numpy.datetime64('2005-04-08T00:00:00.000000000'),
  'end_date': numpy.datetime64('2005-06-03T00:00:00.000000000'),
  'min_point': 818.03,
  'max_point': 1003.45},
 {'state': 'bull',
  'start_date': numpy.datetime64('2005-06-03T00:00:00.000000000'),
  'end_date': numpy.datetime64('2007-10-16T00:00:00.000000000'),
  'min_point': 818.03,
  'max_point': 5877.2},
 {'state': 'bear',
  'start_date': numpy.datetime64('2007-10-16T00:00:00.000000000'),
  'end_date': numpy.datetime64('2007-11-28T00:00:00.000000000'),
  'min_point': 4648.75,
  'max_point': 5877.2},
 {'state': 'bull',
  'start_date': numpy.datetime64('2007-11-28T00:00:00.000000000'),
  'end_date': numpy.datetime64('2008-01-14T00:00:00.000000000'),
  'min_point': 4648.75,
  'max_point': 5731.76},
 {'state': 'bear',
  'start_date': numpy.datetime64('2008-01-14T00:00:00.000000000'),
  'end_date': numpy.datetime64('2008-04-21T00:00:00.000000000'),
  'min_point': 3267.55,
  'max_point': 5731.76},
 {'s

In [36]:
s_x, s_y = [], []
for regime in market_regimes:
    if regime["state"] == "bull":
        s_y.extend([regime["min_point"], regime["max_point"]])
        s_x.extend([pd.to_datetime(regime["start_date"]), pd.to_datetime(regime["end_date"])])
    else:
        s_y.extend([regime["max_point"], regime["min_point"]])
        s_x.extend([pd.to_datetime(regime["start_date"]), pd.to_datetime(regime["end_date"])])

In [37]:
s_x

[Timestamp('2005-04-08 00:00:00'),
 Timestamp('2005-06-03 00:00:00'),
 Timestamp('2005-06-03 00:00:00'),
 Timestamp('2007-10-16 00:00:00'),
 Timestamp('2007-10-16 00:00:00'),
 Timestamp('2007-11-28 00:00:00'),
 Timestamp('2007-11-28 00:00:00'),
 Timestamp('2008-01-14 00:00:00'),
 Timestamp('2008-01-14 00:00:00'),
 Timestamp('2008-04-21 00:00:00'),
 Timestamp('2008-04-21 00:00:00'),
 Timestamp('2008-05-05 00:00:00'),
 Timestamp('2008-05-05 00:00:00'),
 Timestamp('2008-11-04 00:00:00'),
 Timestamp('2008-11-04 00:00:00'),
 Timestamp('2009-08-03 00:00:00'),
 Timestamp('2009-08-03 00:00:00'),
 Timestamp('2009-08-31 00:00:00'),
 Timestamp('2009-08-31 00:00:00'),
 Timestamp('2009-12-07 00:00:00'),
 Timestamp('2009-12-07 00:00:00'),
 Timestamp('2010-07-05 00:00:00'),
 Timestamp('2010-07-05 00:00:00'),
 Timestamp('2010-11-08 00:00:00'),
 Timestamp('2010-11-08 00:00:00'),
 Timestamp('2012-12-03 00:00:00'),
 Timestamp('2012-12-03 00:00:00'),
 Timestamp('2013-02-06 00:00:00'),
 Timestamp('2013-02-

In [42]:
trace1 = go.Scatter(
    x = csi_300.index,
    y = csi_300["close"],
    mode = "lines",
    name = "CSI 300"
)

trace2 = go.Scatter(
    x = s_x,
    y = s_y,
    mode = "markers",
    name = "CSI 300"
)


data = [trace1, trace2]

layout = go.Layout(
    legend = {"x": 1, "y": 1},
    title = "CSI Switch Points"
)

fig = go.Figure(data=data, layout = layout)

plotly.offline.init_notebook_mode()
plotly.offline.iplot(fig, filename='scatter-mode')

AttributeError: 'NoneType' object has no attribute 'index'